# Path in a graph
## Understanding via visualization
#### Nathaniel Raimbault, May 2019

# Aim of this app
The aim of this app is to:
- Draw links between nodes based on a (2D) distance threshold
- Visualize whether a path exists between two given nodes


Read the _tasks_ below and play with the sliders to see the result!

# Find a path between 2 points
Given the coordinates of a set of points on the plane, we consider a link between two points
if their distance is below a given threshold (obtaining in this way an undirected graph).
Then, for a given pair of points, we want to know whether this pair is connected or not through a path in the graph.

# Tasks
- <span style="color: #cc0000">Move the sliders below and look at the results.</span>
- <span style="color: #cc0000">For a given number of points, see how the number of links varies with the threshold value.</span>
- <span style="color: #cc0000">See how the probability of finding a path between two nodes depends on the number of links and the number of points (the size of the box is fixed)</span>
- <span style="color: #cc0000">Understand the algorithm that determines whether a path exists</span>


# Visualization
With the following selectors, you can pick:
- the number $N$ of points in your graph
- the distance threshold for determining whether or not a link exists between 2 points
- the pair of point you want to find a path for

In [1]:
import math
import numpy as np
import bqplot.pyplot as pl
import json
import time

In [2]:
def get_dist_vect(p_, q_): # Calculates distance between 2 points
    return np.sqrt(np.sum((p_-q_)**2,axis=2))

def generate_pts(number_of_points,max_spatial_extent):
    pts = np.random.uniform(-max_spatial_extent,max_spatial_extent, size=(number_of_points,2))
    return pts

def get_connect(pts,threshold,pairs,algo_type):

    N = len(pts)
    
    # 1) Find all links    
    
    # Calculate all distances between all points and store them into a matrix, the indices of which will correspond to the points' ids
    i_, j_ = np.ogrid[:N,:N]
    dist_mat = get_dist_vect(pts[i_],pts[j_])
    
    # Create list of links which will contain all the links between points.
    links_list = [[i] for i in range(N)] # Initially, we just know each node is linked with itself
    
    # Loop over matrix of distances and check whether or not its elements are below the threshold
    for i_,row in enumerate(dist_mat):
        for j_,col in enumerate(row[i_:]):
            if (col < threshold and col > 0):
                links_list[i_].append(i_+j_) # Add id of the connected point...
                links_list[i_+j_].append(i_) # ...and its symmetric counterpart
    
    # 2) Find whether path exists
       
    booleans_list = [] # Will contain a list of booleans, telling if a path exists between 2 given points
    
    # Run through all given pairs of points
    for pair in pairs:
        frontier = [pair[0]] # Will correspond to all the points to explore at a given step of the procedure. Here initialized to the first element of the pair
        all_nodes = [] # Will contain all the nodes already visited
        seg_visited_x = []
        seg_visited_y = []
        found = False # Boolean to state if there is a path between 2 nodes or not
    
        while (frontier): # Continue as long as "frontier" is not empty
            next_ = [] # Will contain the points to explore for the next step of the procedure
            # Loop over the points in "frontier"
            for u in frontier:
              # Check if the second element of the pair is present in the points linked to u
                if (pair[1] in links_list[u]):
                    found = True
                    next_= []
                    all_nodes.append(pair[0])
                    # I use the "None trick" to plot segments fast.
                    #For some reason, it stopped working with bqplot, but it works with np.nan instead of None.
                    seg_visited_x = np.append(seg_visited_x,[pts[u,0],pts[pair[1],0],np.nan]) # Order of the segments visited
                    seg_visited_y = np.append(seg_visited_y,[pts[u,1],pts[pair[1],1],np.nan]) # Order of the segments visited 
                    break
                else:
                    # Loop over the points linked to "u"
                    for v in links_list[u]:
                        seg_visited_x = np.append(seg_visited_x,[pts[u,0],pts[v,0],np.nan]) # Order of the segments visited
                        seg_visited_y = np.append(seg_visited_y,[pts[u,1],pts[v,1],np.nan]) # Order of the segments visited  
                        if (v not in all_nodes): # Avoid replicas by adding only points not yet visited
                            all_nodes.append(v) # Keep a trace of all the nodes we have visited
                            next_.append(v) # At the next iteration we will explore the points we have not explored yet
            frontier = next_
      
        booleans_list.append(found)
    
    return links_list,all_nodes,booleans_list,seg_visited_x,seg_visited_y

In [3]:
from ipywidgets import Accordion, IntSlider, FloatSlider, HTMLMath, Dropdown, Box, HBox, VBox, Layout
from IPython.display import display

n_widget = IntSlider(value=20, min=2, max=200, description = "$N$", continuous_update=False)

n_threshold_widget = FloatSlider(value=0.0, min=0, max=600, description = r"$\text{threshold}$", continuous_update=False)
n_threshold_widget2 = FloatSlider(value=0.0, min=0, max=600, description = r"$\text{threshold}$", continuous_update=False)

type_widget = Dropdown(options=(
        ("Algorithm 1","algorithm1"),
    ), 
    description = "Algorithm type", continuous_update=False, layout=Layout(width='250px'))
n_pair1_widget = IntSlider(value=0, min=0, max=n_widget.value-1, description = r"First node", continuous_update=False)
n_pair2_widget = IntSlider(value=1, min=0, max=n_widget.value-1, description = r"Second node", continuous_update=False)

result_plot = pl.figure()
result_plot2 = pl.figure()
graph_description = HTMLMath(value="")
graph_description2 = HTMLMath(value="")

def update_pairs_range(*args):
    n_pair1_widget.max = n_widget.value-1
    n_pair2_widget.max = n_widget.value-1
n_widget.observe(update_pairs_range, 'value')




In [4]:
#On call, this function will generate a set of points and store them as a global variable passed in several other functions
def on_func_change(change):
    global pt
    pt = generate_pts(
        number_of_points=n_widget.value, 
        max_spatial_extent=500)
    on_graph_params_change_nopath(None)
n_widget.observe(on_func_change)
# on_func_change(None)

In [5]:
def on_graph_params_change_nopath(change):
#     global pt
#     pt = generate_pts(
#         number_of_points=n_widget.value, 
#         max_spatial_extent=500)
    
    links_list, _, booleans_list, _,_ = get_connect(
        pt, 
        threshold=n_threshold_widget.value,
        pairs=[[n_pair1_widget.value,n_pair2_widget.value]],
        algo_type=type_widget.value)
    
    segments_x = []
    segments_y = []
    for i,neighbours in enumerate(links_list):
        for node in neighbours[1:]:
            segments_x = np.append(segments_x,[pt[i,0],pt[node,0],np.nan])
            segments_y = np.append(segments_y,[pt[i,1],pt[node,1],np.nan])
    
    description_string = (r"<strong>N</strong>: number of nodes<br>"
            r"<strong>threshold</strong>: distance threshold for determining whether 2 nodes have a direct link<br>")

    graph_description.value = description_string
    pl.figure(fig=result_plot)
    result_plot.legend_location = 'top-left'
    pl.clear()
    #reverse_options_map = {_[1]: _[0] for _ in type_widget.options}
    pl.title('Connectivity graph')
    pl.legend()
    
    #pl.plot(pt[:,0],pt[:,1], 'ro',labels=["pts"], colors=["#ff0000"])
    scatt=pl.scatter(pt[:,0],pt[:,1], labels=["pts"], default_size=150,colors=["white"],stroke='black',names=[str(j) for j,_ in enumerate(pt)])
    global seg_pl0
    seg_pl0=pl.plot(segments_x,segments_y,labels=["links"])
    #seg_pl=pl.plot([100,200,None,5,90],[100,300,None,200,400])
    #return seg_pl0

def on_segments_change0(change):
    links_list, _, booleans_list,_,_ = get_connect(
        pt, 
        threshold=n_threshold_widget.value,
        pairs=[[0,0]],
        algo_type=type_widget.value)

    segments_x = []
    segments_y = []
    for i,neighbours in enumerate(links_list):
        for node in neighbours[1:]:
            segments_x = np.append(segments_x,[pt[i,0],pt[node,0],np.nan])
            segments_y = np.append(segments_y,[pt[i,1],pt[node,1],np.nan])
    
    seg_pl0.x=segments_x
    seg_pl0.y=segments_y

#n_widget.observe(on_graph_params_change_nopath, names='value', type='change')
#n_threshold_widget.observe(on_graph_params_change_nopath, names='value', type='change')
n_threshold_widget.observe(on_segments_change0, names='value', type='change')

# Create the plot
#on_func_change(None)
#on_graph_params_change_nopath(None)

In [6]:
# Create the plot
on_func_change(None)

In [7]:
display(Box([
        VBox([n_widget,n_threshold_widget], layout=Layout(width='350px')),
        VBox([graph_description], layout=Layout(min_width='300px')),
    ], layout=Layout(width='100%', flex_flow='row wrap', display='flex')))

Box(children=(VBox(children=(IntSlider(value=20, continuous_update=False, description='$N$', max=200, min=2), …

In [8]:
result_plot.layout.min_width = '800px'
result_plot.layout.max_width = '800px'
result_plot.layout.min_height = '800px'
result_plot.layout.max_height = '800px'
display(Box(children=[result_plot], layout=Layout(justify_content='center')))

Box(children=(Figure(axes=[Axis(scale=LinearScale()), Axis(orientation='vertical', scale=LinearScale())], fig_…

# Find a path
Now you (well, at least I ;)) want to know if a path exists between 2 nodes. This is taken care of by the second part of the function _get_connect_ defined earlier.

We define almost the same plotting function as before, but now we color progressively the nodes explored by the algorithm.
Move the cursors to relaunch the animation.

In [9]:
def on_graph_params_change(change):
    links_list, all_nodes,booleans_list,seg_visited_x,seg_visited_y = get_connect(
        pt, 
        threshold=n_threshold_widget2.value,
        pairs=[[n_pair1_widget.value,n_pair2_widget.value]],
        algo_type=type_widget.value)
    
    segments_x = []
    segments_y = []
    for i,neighbours in enumerate(links_list):
        for node in neighbours[1:]:
            segments_x = np.append(segments_x,[pt[i,0],pt[node,0],np.nan])
            segments_y = np.append(segments_y,[pt[i,1],pt[node,1],np.nan])
    
    description_string = (
            r"<strong>threshold</strong>: distance threshold for determining whether 2 nodes have a direct link<br>"
            r"<strong>First node, second node</strong>: pair of nodes you want to determine a path for<br>")

    graph_description2.value = description_string
    pl.figure(fig=result_plot2)
    result_plot.legend_location = 'top-left'
    pl.clear()
    reverse_options_map = {_[1]: _[0] for _ in type_widget.options}
    pl.title('Connectivity graph')
    pl.legend()
    
    #pl.plot(pt[:,0],pt[:,1], 'ro',labels=["pts"], colors=["#ff0000"])
    scatt=pl.scatter(pt[:,0],pt[:,1], labels=["pts"], default_size=150,colors=["white"],stroke='black',names=[str(j) for j,_ in enumerate(pt)])
    global seg_pl
    seg_pl=pl.plot(segments_x,segments_y,labels=["links"])
    #seg_pl=pl.plot([100,200,None,5,90],[100,300,None,200,400])
    #pl.plot(seg_visited_x,seg_visited_y,labels=["links"],colors=["green"])
#     for i in range(0,len(seg_visited_x),3):
#         pl.plot([seg_visited_x[i:i+2]],[seg_visited_y[i:i+2]],labels=["links"],colors=["green"])
#         #scatt2=pl.scatter([pt[i,0]],[pt[i,1]],default_size=150, colors=["green"])
#         time.sleep(1/n_widget.value)
        
    for i,node in enumerate(all_nodes):
        scatt2=pl.scatter([pt[node,0]],[pt[node,1]],default_size=150, colors=["green"])
        time.sleep(15/n_widget.value)
    if (booleans_list[0]):
        pl.scatter([pt[n_pair2_widget.value,0]],[pt[n_pair2_widget.value,1]],default_size=150,colors=["green"])
        pl.label(["Path found between nodes {} and {} :-)".format(str(n_pair1_widget.value),str(n_pair2_widget.value))],x=[-200],y=[0],colors=["black"])
    else:
        pl.label(["No path found between nodes {} and {} :-(".format(str(n_pair1_widget.value),str(n_pair2_widget.value))],x=[-200],y=[0],colors=["black"])

# def on_segments_change(change):
#     links_list, _, booleans_list = get_connect(
#         pt, 
#         threshold=n_threshold_widget2.value,
#         pairs=[[0,0]],
#         algo_type=type_widget.value)

#     segments_x = []
#     segments_y = []
#     for i,neighbours in enumerate(links_list):
#         for node in neighbours[1:]:
#             segments_x = np.append(segments_x,[pt[i,0],pt[node,0],None])
#             segments_y = np.append(segments_y,[pt[i,1],pt[node,1],None])
    
#     seg_pl.x=segments_x
#     seg_pl.y=segments_y
        
n_widget.observe(on_graph_params_change, names='value', type='change')
#n_threshold_widget2.observe(on_segments_change, names='value', type='change')
n_threshold_widget2.observe(on_graph_params_change, names='value', type='change')
type_widget.observe(on_graph_params_change, names='value', type='change')
n_pair1_widget.observe(on_graph_params_change, names='value', type='change')
n_pair2_widget.observe(on_graph_params_change, names='value', type='change')
# Create the plot
on_graph_params_change(None)

In [10]:
display(Box([
        VBox([n_threshold_widget2,n_pair1_widget,n_pair2_widget, type_widget], layout=Layout(width='350px')),
        VBox([graph_description2], layout=Layout(min_width='300px')),
    ], layout=Layout(width='100%', flex_flow='row wrap', display='flex')))

Box(children=(VBox(children=(FloatSlider(value=0.0, continuous_update=False, description='$\\text{threshold}$'…

In [11]:
result_plot2.layout.min_width = '800px'
result_plot2.layout.max_width = '800px'
result_plot2.layout.min_height = '800px'
result_plot2.layout.max_height = '800px'
display(Box(children=[result_plot2], layout=Layout(justify_content='center')))

Box(children=(Figure(axes=[Axis(scale=LinearScale()), Axis(orientation='vertical', scale=LinearScale())], fig_…

# Different algorithms
You can now try to understand how the algorithm works. You can do this in several ways:
- By looking directly at the function get_connect
- By looking at the series of illuminated green nodes and deduce the pattern
- By looking at the reference provided. This algorithm is known as breadth-first search (BFS).

You have seen that the Dropdown menu contains a single entry called "Algorithm 1".
If you have understood how the present algorithm works, you can think of another way to find a path between nodes. Alternatively, you could also try to implement an already-existing algorithm, and add it to the list of possible algorithms, and compare which one is the faster to find a path.

Such algorithms have many practical applications (road networks -google map-, path-findings in video games, Rubik's cube solver, etc.)


# References
[1] https://en.wikipedia.org/wiki/Breadth-first_search

In [12]:
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))